In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Armado del dataset

#### Votos validos

In [ ]:
df = pd.read_csv('presidente_afirmativos.csv')

In [ ]:
df.shape

#### Mesas

In [ ]:
total_data_mesas = pd.read_csv('total_data_mesas.csv')

In [ ]:
total_data_mesas.shape

## Distribucion de mesas por % sacado

In [ ]:
for a, a_df in df.groupby('agrupacion_nombre'):
    fig = plt.figure(figsize=(15, 8))
    plt.title(a)
    a_df['percentage'].hist(bins=100)

## Mesas con % de votos de un partido

In [ ]:
set_percentage = 0.9

In [ ]:
mesas_percentage = total_data_mesas[total_data_mesas['percentage'] >= set_percentage]

#### Cantidad de mesas por agrupacion

In [ ]:
mesas_percentage['agrupacion_ganadora'].value_counts()

In [ ]:
mesas_percentage[mesas_percentage['agrupacion_ganadora']=='UNION POR LA PATRIA'].sort_values(
    'percentage', ascending=False)

#### Cantidad de mesas por agrupacion y distrito

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_ganadora', 'distrito_nombre']).count().sort_values(
    ['agrupacion_ganadora', 'votos_cantidad'], ascending=False)

#### Cantidad de votos por agrupacion

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'votos_cantidad']].groupby(
    'agrupacion_ganadora').sum().sort_values('votos_cantidad', ascending=False)

#### Cantidad de mesas por agrupacion y distrito

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_ganadora', 'distrito_nombre']).sum().sort_values(
    ['agrupacion_ganadora', 'votos_cantidad'], ascending=False)

## Porcentaje de victoria

#### Total

In [ ]:
percentage_dfs = []

In [ ]:
for p, p_df in total_data_mesas.groupby('agrupacion_ganadora'):
    temp_df = p_df['percentage'].describe().to_frame()
    temp_df.columns = [p]
    percentage_dfs.append(temp_df.T)

In [ ]:
pd.concat(percentage_dfs)

## Votos no validos

#### % de mesas sin invalidos por agrupacion ganadora

In [ ]:
zero_invalidos = total_data_mesas[total_data_mesas['total_invalidos'] == 0].copy()

In [ ]:
agrupacion_grp_sin = zero_invalidos['agrupacion_ganadora'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_sin.columns = ['agrupacion_ganadora', 'sin_invalidos']

In [ ]:
agrupacion_grp_total = total_data_mesas['agrupacion_ganadora'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_total.columns = ['agrupacion_ganadora', 'total']

In [ ]:
agrupaciones_ganadoras_invalidos = pd.merge(agrupacion_grp_sin, agrupacion_grp_total)

In [ ]:
agrupaciones_ganadoras_invalidos['percentage'] = agrupaciones_ganadoras_invalidos[
    'sin_invalidos'] / agrupaciones_ganadoras_invalidos['total']

In [ ]:
agrupaciones_ganadoras_invalidos.sort_values('percentage', ascending=False)

#### Mesas ganadores con mas invalidos por agrupacion

In [ ]:
dfs = []

In [ ]:
for p, p_df in total_data_mesas.groupby('agrupacion_ganadora'):
    temp_df = p_df['invalidos_percentage'].describe().to_frame()
    temp_df.columns = [p]
    dfs.append(temp_df.T)

In [ ]:
inv_dist_df = pd.concat(dfs)

In [ ]:
inv_dist_df

In [ ]:
for v in ['mean', '25%', '50%', '75%', 'max']:
    fig = plt.figure(figsize=(15, 8))
    plt.title(v)
    plot_df = inv_dist_df.sort_values(v)
    x_values = plot_df.index
    plt.bar(x_values, plot_df[v])

## Mesas con 0 votos

In [ ]:
partido = 'LA LIBERTAD AVANZA'

In [ ]:
zero_votos = votos_por_mesa[votos_por_mesa[partido] == 0]

In [ ]:
zero_votos['total_invalidos'].value_counts()

In [ ]:
zero_votos[zero_votos['total_invalidos']==96]